In [1]:
import pandas as pd

full_data = pd.read_csv("everything100percat.csv")

In [2]:

def calibrate_thresholds(
    df: pd.DataFrame,
    baseline_mask: pd.Series,
    features_params: dict
) -> dict:
    """
    Compute trigger thresholds for each feature based on a baseline window.

    Args:
      df             : full DataFrame with features
      baseline_mask  : boolean Series marking baseline rows (e.g., pre‑ChatGPT)
      features_params: dict mapping feature names to:
                       {
                         "direction": "greater" or "less",
                         "percentile": e.g. 95 (for greater) or 5 (for less)
                       }
    Returns:
      thresholds: dict mapping feature to {"threshold": float, "direction": str}
    """
    baseline_df = df.loc[baseline_mask]
    thresholds = {}
    for feat, params in features_params.items():
        pct = params.get("percentile", 95 if params["direction"]=="greater" else 5) / 100
        if params["direction"] == "greater":
            thr = baseline_df[feat].quantile(pct)
        else:
            thr = baseline_df[feat].quantile(pct)
        thresholds[feat] = {"threshold": thr, "direction": params["direction"]}
    return thresholds

def rule_based_flag(
    row: pd.Series,
    thresholds: dict,
    min_votes: int
) -> bool:
    """
    Apply rule‑based voting for a single revision.

    Args:
      row        : one DataFrame row of features
      thresholds : dict from calibrate_thresholds()
      min_votes  : minimum number of triggered rules to label as AI

    Returns:
      True if AI‑assisted (votes >= min_votes), else False
    """
    votes = 0
    for feat, params in thresholds.items():
        val = row.get(feat, None)
        if val is None or pd.isna(val):
            continue
        if params["direction"] == "greater" and val > params["threshold"]:
            votes += 1
        elif params["direction"] == "less" and val < params["threshold"]:
            votes += 1
    return votes >= min_votes

def apply_rule_based_flags(
    df: pd.DataFrame,
    thresholds: dict,
    min_votes: int
) -> pd.Series:
    """
    Vectorized application: returns a boolean Series of AI flags.
    """
    return df.apply(lambda r: rule_based_flag(r, thresholds, min_votes), axis=1)

# --- Example Usage ---

# 1) Define which features to include and how to calibrate
features_params = {
    "lexical_spike_delta": {"direction": "greater", "percentile": 95},
    "perplexity":          {"direction": "less",    "percentile": 5},
    "citation_delta":      {"direction": "less",    "percentile": 5},
    "burstiness":          {"direction": "less",    "percentile": 5},
    "nTTR":                {"direction": "less",    "percentile": 5},
}

# 2) Identify baseline window (e.g., pre‑ChatGPT)
baseline_mask = full_data["timestamp"] < "2022-11-01"

# 3) Calibrate thresholds from baseline
thresholds = calibrate_thresholds(full_data, baseline_mask, features_params)

# 4) Choose minimum votes (e.g., 2 out of 5 signals)
min_votes = 2

# 5) Flag revisions
full_data["ai_flag"] = apply_rule_based_flags(full_data, thresholds, min_votes)

# Inspect thresholds and AI‐flagged count
print("Thresholds per feature:", thresholds)
print("AI‐flagged revisions:", full_data["ai_flag"].sum())



Thresholds per feature: {'lexical_spike_delta': {'threshold': np.float64(0.03160831639765665), 'direction': 'greater'}, 'perplexity': {'threshold': np.float64(219.4595947265625), 'direction': 'less'}, 'citation_delta': {'threshold': np.float64(0.0006215535705276602), 'direction': 'less'}, 'burstiness': {'threshold': np.float64(0.0584613878072789), 'direction': 'less'}, 'nTTR': {'threshold': np.float64(5.700198591836499), 'direction': 'less'}}
AI‐flagged revisions: 654


In [3]:
# Save the results
full_data.to_csv("everything100percat_with_ai_flags.csv", index=False)